# Coronavirus (COVID-19) Dashboard

For this project, I was interested in comparing specific COVID-19 metrics, specifically cases, deaths and vaccinations) and how they varied over the last few years (Jan 1, 2020 - Nov 2, 2023). The data for this project was sourced from [Our World in Data](https://ourworldindata.org/coronavirus).

In [1]:
# --- load necessary libraries ---
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
import datetime as dt
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.formatters import NumeralTickFormatter
pd.options.plotting.backend = 'holoviews'
pn.extension()

In [2]:
# --- set formatting for all plots ---
formatter_date = DatetimeTickFormatter(months = '%b %Y')
formatter_num = NumeralTickFormatter(format = "0,0")

grid_style = {'ygrid_line_dash': [4, 4],
              'ygrid_line_color': 'grey',
              'xgrid_line_color': 'white'}

## Data import and preprocessing

In this section, I uploaded and quickly explored the data. I selected only the columns I would need and did some minor preprocessing such as, changing the *date column* to the appropriate *datetime* data type.

In [3]:
# --- load dataset from https://ourworldindata.org/covid-deaths ---
df = pd.read_csv('owid-covid-data.csv',
                 parse_dates = ['date'], 
                 date_format = '%Y-%m-%d')

# view a sample of the dataset
df.sample(5)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
159843,KEN,Africa,Kenya,2021-02-21,103993.0,152.0,171.571,1817.0,4.0,3.143,...,20.4,24.651,1.40,66.70,0.601,54027484.0,NaN,NaN,NaN,NaN
146521,IRQ,Asia,Iraq,2022-12-31,2465373.0,0.0,39.714,25374.0,0.0,0.143,...,NaN,94.576,1.40,70.60,0.674,44496124.0,NaN,NaN,NaN,NaN
13446,ARG,South America,Argentina,2022-04-26,9072230.0,0.0,1615.286,128542.0,0.0,28.286,...,27.7,NaN,5.00,76.67,0.845,45510324.0,NaN,NaN,NaN,NaN
217174,NLD,Europe,Netherlands,2022-08-09,8355339.0,0.0,2265.143,22543.0,0.0,4.429,...,27.3,NaN,3.32,82.28,0.944,17564020.0,NaN,NaN,NaN,NaN
37030,BOL,South America,Bolivia,2021-09-22,497386.0,286.0,302.000,18664.0,5.0,9.857,...,NaN,25.383,1.10,71.51,0.718,12224114.0,NaN,NaN,NaN,NaN


In [4]:
# --- view size of dataset ---
df.shape

(353698, 67)

The dataset contains **67 features/columns** and **353,698 observations/rows**.

In [5]:
# --- view dataset columns ---
df.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [6]:
# --- select pertinent features/columns ---
df = df[['continent',
         'location',
         'date',
         'total_cases',
         'new_cases',
         'total_deaths',
         'new_deaths',
         'people_vaccinated',
         'people_fully_vaccinated']]

In [7]:
# --- replace nas with 0 ---
df = df.fillna(0)

## Create the dashboard

In this section, I first converted the dataframe into an interactive data frame, created the **Year slider** to allow selection of data up to a specific year. I then renamed the columns to allow creation of well-displayed widgets.

In [8]:
# --- make dataframe interactive ---
int_df = df.interactive()

In [9]:
# --- create year slider widget for dashboard ---
year_slider = pn.widgets.IntSlider(name = 'Year slider',
                                   start = 2020, # min year in dataset
                                   end = 2023, # max year in dataset
                                   step = 1,
                                   value = 2021)
year_slider

IntSlider(end=2023, name='Year slider', start=2020, value=2021)

In [10]:
# rename column options
int_df = int_df.rename(
    columns = {
        'date': 'Date',
        'total_cases': 'Cumulative Total Cases', 
        'new_cases': 'New Cases',
        'total_deaths': 'Cumulative Total Deaths',
        'new_deaths': 'New Deaths',
        'people_vaccinated': 'People Vaccinated',
        'people_fully_vaccinated': 'People Fully Vaccinated',
        'location': 'Location'})

In [11]:
# --- create radiobuttons ---

# for total and new cases
cases = pn.widgets.RadioButtonGroup(
    name = 'Cases',
    options = ['Cumulative Total Cases',
               'New Cases'],
    button_style = 'solid',
    orientation = 'vertical',
    button_type = 'light'
)

# for total and new deaths
deaths = pn.widgets.RadioButtonGroup(
    name = 'Deaths',
    options = ['Cumulative Total Deaths',
               'New Deaths'],
    button_style = 'solid',
    orientation = 'vertical',
    button_type = 'light'
)

# for people vaccinated & people fully vaccinated
vaccinations = pn.widgets.RadioButtonGroup(
    name = 'Vaccinations',
    options = ['People Vaccinated',
               'People Fully Vaccinated'],
    button_style = 'solid',
    orientation = 'vertical',
    button_type = 'light'
)

### Worldwide COVID Status

In this section, I created the pipelines and plots using global COVID-19 data.

In [12]:
# --- create world data pipelines ---

# cases
world_cases_pipeline = (
    int_df[
        (int_df['Date'].dt.year <= year_slider) &
        (int_df['Location'] == 'World')]
    .groupby(['Location',
              'Date'])[cases].sum())

# deaths
world_deaths_pipeline = (
    int_df[
        (int_df['Date'].dt.year <= year_slider) &
        (int_df['Location'] == 'World')]
    .groupby(['Location',
              'Date'])[deaths].sum())

# vaccinations
world_vaccinations_pipeline = (
    int_df[
        (int_df['Date'].dt.year <= year_slider) &
        (int_df['Location'] == 'World')]
    .groupby(['Location',
              'Date'])[vaccinations].sum())

In [13]:
# --- create world data plots ---

# cases
world_cases_plot = world_cases_pipeline.hvplot.line(
    x = 'Date',
    y = cases,
    line_width = 1,
    title = '',
    xlabel = '',
    ylabel = cases,
    responsive = True,
    fontsize = {'ylabel': 11, 
                'ticks': 9}).opts(autorange = 'y',
                                  height = 350,
                                  gridstyle = grid_style,
                                  show_grid = True,
                                  shared_axes = False,
                                  xformatter = formatter_date,
                                  yformatter = formatter_num).output()

# deaths
world_deaths_plot = world_deaths_pipeline.hvplot.line(
    x = 'Date',
    y = deaths,
    line_width = 1,
    title = '',
    xlabel = '',
    ylabel = deaths,
    responsive = True,
    fontsize = {'ylabel': 11, 
                'ticks': 9}).opts(autorange = 'y',
                                  height = 350,
                                  gridstyle = grid_style,
                                  show_grid = True,
                                  shared_axes = False,
                                  xformatter = formatter_date,
                                  yformatter = formatter_num).output()

# vaccinations
world_vaccinations_plot = world_vaccinations_pipeline.hvplot.line(
    x = 'Date',
    y = vaccinations,
    line_width = 1,
    title = '',
    xlabel = '',
    ylabel = vaccinations,
    responsive = True,
    fontsize = {'ylabel': 11, 
                'ticks': 9}).opts(autorange = 'y',
                                  height = 350,
                                  gridstyle = grid_style,
                                  show_grid = True,
                                  shared_axes = False,
                                  xformatter = formatter_date,
                                  yformatter = formatter_num).output()

### COVID Status by Continent

In this section, I created pipelines and plots for data on the level of the continents.

In [14]:
# --- create list of all continents ---
continents = ['Asia',
              'Europe',
              'Africa',
              'Oceania',
              'North America',
              'South America']

In [15]:
# --- create continent data pipelines ---

# cases
continent_cases_pipeline = (
    int_df[
        (int_df['Date'].dt.year <= year_slider) &
        (int_df['Location'].isin(continents))]
    .groupby(['Location',
              'Date'])[cases].sum())

# deaths
continent_deaths_pipeline = (
    int_df[
        (int_df['Date'].dt.year <= year_slider) &
        (int_df['Location'].isin(continents))]
    .groupby(['Location',
              'Date'])[deaths].sum())

# vaccinations
continent_vaccinations_pipeline = (
    int_df[
        (int_df['Date'].dt.year <= year_slider) &
        (int_df['Location'].isin(continents))]
    .groupby(['Location',
              'Date'])[vaccinations].sum())

In [16]:
# --- create continent data plots ---

# cases
continent_cases_plot = continent_cases_pipeline.hvplot.line(
    x = 'Date',
    y = cases,
    line_width = 1,
    by = 'Location',
    title = '',
    xlabel = '',
    ylabel = cases,
    responsive = True,
    fontsize = {'ylabel': 11, 
                'ticks': 9}).opts(autorange = 'y',
                                  height = 350,
                                  gridstyle = grid_style,
                                  show_grid = True,
                                  xformatter = formatter_date,
                                  yformatter = formatter_num,
                                  legend_position='top_left').output()

# deaths
continent_deaths_plot = continent_deaths_pipeline.hvplot.line(
    x = 'Date',
    y = deaths,
    line_width = 1,
    by = 'Location',
    title = '',
    xlabel = '',
    ylabel = deaths,
    responsive = True,
    fontsize = {'ylabel': 11, 
                'ticks': 9}).opts(autorange = 'y',
                                  height = 350,
                                  gridstyle = grid_style,
                                  show_grid = True,
                                  xformatter = formatter_date,
                                  yformatter = formatter_num,
                                  legend_position='top_left').output()

# vaccinations
continent_vaccinations_plot = continent_vaccinations_pipeline.hvplot.line(
    x = 'Date',
    y = vaccinations,
    line_width = 1,
    by = 'Location',
    title = '',
    xlabel = '',
    ylabel = vaccinations,
    responsive = True,
    fontsize = {'ylabel': 11, 
                'ticks': 9}).opts(autorange = 'y',
                                  height = 350,
                                  gridstyle = grid_style,
                                  show_grid = True,
                                  xformatter = formatter_date,
                                  yformatter = formatter_num,
                                  legend_position='top_left').output()

### Format dashboard layout

In this section, I used the FastListTemplate template to format the dashboard layout.

In [17]:
# --- create template for dashboard ---

# world plots
world = pn.Column(pn.Column('### CASES',
                            pn.Row(cases,
                                   world_cases_plot)),
                  pn.Column('### DEATHS',
                            pn.Row(deaths,
                                   world_deaths_plot)),
                  pn.Column('### VACCINATIONS',
                            pn.Row(vaccinations,
                                   world_vaccinations_plot)))


# continent plots
continent = pn.Column(pn.Column('### CASES',
                                pn.Row(cases,
                                       continent_cases_plot)),
                      pn.Column('### DEATHS',
                                pn.Row(deaths,
                                       continent_deaths_plot)),
                      pn.Column('### VACCINATIONS',
                                pn.Row(vaccinations,
                                       continent_vaccinations_plot)))

# create dashboard template
template = pn.template.FastListTemplate(
    title = '<b>Coronavirus (COVID-19) Dashboard</b>',
    sidebar = [pn.pane.Markdown('## COVID-19 Status'),
               pn.pane.JPG('cdc-w9KEokhajKw-unsplash.jpg',
                           sizing_mode = 'scale_both'),
               pn.pane.Markdown('This dashboard shows an overview of a few crucial COVID-19 metrics monitored from 2020-2023'),
               pn.pane.Markdown('## \n ## Year Slider'),
               year_slider],
    main = [pn.Tabs(('World COVID 19 Statistics',
                      world),
                    ('COVID 19 Statistics by Continent',
                      continent),
                    sizing_mode = 'scale_both',
                   background = '#DDDDDD')],
    theme_toggle = False,
    sidebar_footer = '<br><font size="1"><b>Data source:</b> Our World in Data (https://ourworldindata.org/coronavirus)</font>',
    header_background = '#002F6C'
)

template.show()
# template.servable;

Launching server at http://localhost:63267


/var/folders/tx/2p6_fhrx5zgdh6s7rky7wbth0000gn/T/ipykernel_12046/280282470.py:35: PanelDeprecationWarning: "Tabs(..., background='#DDDDDD')" is deprecated and will be removed in version 1.3, use "Tabs(..., styles={'background': '#DDDDDD'})" instead.
  main = [pn.Tabs(('World COVID 19 Statistics',
